In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import country_converter as coco

In [3]:
df = pd.read_csv("./data/countries.csv")
agro = pd.read_csv("./data/Agroalimentaire/global-food-prices.csv")

/tmp/ipykernel_4578/4114859036.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  agro = pd.read_csv("./data/Agroalimentaire/global-food-prices.csv")


In [4]:
def perf_coco(df2):
    """
    Fonction perf_coco -
    Paramètre : 
    - df2 : colonne de nom de pays à valeur unique

    Sortie : Comparaison entre aucun changement, coco_convert en name short et coco_convert en ISO2
    """
    print("===== Performance sans changement : ======")
    set_pays = set(df["Name"])
    set_df2 = set(df2)
    valeurs_communes = set_pays.intersection(set_df2)
    print(f"Valeur communes : {len(valeurs_communes)}/{len(df2)}")

    print("===== Performance short name pays : ======")
    set_df2 = set(coco.convert(names=df2,to='name_short'))
    valeurs_communes = set_pays.intersection(set_df2)
    print(f"Valeur communes : {len(valeurs_communes)}/{len(df2)}")

    print("===== Performance code pays : ======")
    set_pays = set(df["Code"])
    set_df2 = set(coco.convert(names=df2,to='ISO2'))
    valeurs_communes = set_pays.intersection(set_df2)
    print(f"Valeur communes : {len(valeurs_communes)}/{len(df2)}")


In [5]:
pays_agro = pd.unique(agro["adm0_name"])
perf_coco(pays_agro)

Bassas da India not found in regex
Bassas da India not found in regex


===== Performance sans changement : ======
Valeur communes : 71/78
===== Performance short name pays : ======
Valeur communes : 64/78
===== Performance code pays : ======
Valeur communes : 77/78


In [6]:
eco = pd.read_csv("./data/Economy/global_gdp.csv")
lst = pd.unique(eco["Entity"])
perf_coco(lst)

East Asia and Pacific (WB) not found in regex
Europe and Central Asia (WB) not found in regex
European Union (27) not found in regex
High-income countries not found in regex
Latin America and Caribbean (WB) not found in regex
Low-income countries not found in regex
Lower-middle-income countries not found in regex
Middle East and North Africa (WB) not found in regex
Middle-income countries not found in regex
North America (WB) not found in regex
South Asia (WB) not found in regex
Sub-Saharan Africa (WB) not found in regex
Upper-middle-income countries not found in regex
World not found in regex
East Asia and Pacific (WB) not found in regex
Europe and Central Asia (WB) not found in regex
European Union (27) not found in regex
High-income countries not found in regex
Latin America and Caribbean (WB) not found in regex


===== Performance sans changement : ======
Valeur communes : 175/208
===== Performance short name pays : ======
Valeur communes : 168/208
===== Performance code pays : ======


Low-income countries not found in regex
Lower-middle-income countries not found in regex
Middle East and North Africa (WB) not found in regex
Middle-income countries not found in regex
North America (WB) not found in regex
South Asia (WB) not found in regex
Sub-Saharan Africa (WB) not found in regex
Upper-middle-income countries not found in regex
World not found in regex


Valeur communes : 192/208


In [7]:
wage = pd.read_csv("./data/Economy/minimum_wage.csv")
perf_coco(wage["country"])

===== Performance sans changement : ======
Valeur communes : 138/154
===== Performance short name pays : ======
Valeur communes : 133/154
===== Performance code pays : ======
Valeur communes : 154/154


In [8]:
## On voit qu'il y le plus de ressemblance grâce à ISO2, on utilisera
## cette standardisation pour la suite.

def standardisation(col):
    """
    Fonction qui prend en paramètre une colonne nom et la transforme en norme ISO2 
    """
    cc = coco.CountryConverter()
    col = cc.convert(col,to = 'ISO3')
    return col

In [9]:
wage["country"] = standardisation(wage["country"])

In [10]:
result = pd.merge(eco,wage,left_on="Code", right_on="country", how="left")
result

,Entity,Code,Year,"GDP per capita, PPP (constant 2017 international $)",country,minimumWageByCountry_annualMinimalWageEarnings,minimumWageByCountry_hourlyMinimalWageEarnings
0,Afghanistan,AFG,2002,1280.4631,AFG,858.0,0.41
1,Afghanistan,AFG,2003,1292.3335,AFG,858.0,0.41
2,Afghanistan,AFG,2004,1260.0605,AFG,858.0,0.41
3,Afghanistan,AFG,2005,1352.3207,AFG,858.0,0.41
4,Afghanistan,AFG,2006,1366.9932,AFG,858.0,0.41
...,...,...,...,...,...,...,...
6557,Zimbabwe,ZWE,2018,2399.6216,NaN,NaN,NaN
6558,Zimbabwe,ZWE,2019,2203.3967,NaN,NaN,NaN
6559,Zimbabwe,ZWE,2020,1990.3195,NaN,NaN,NaN
6560,Zimbabwe,ZWE,2021,2115.1445,NaN,NaN,NaN


In [31]:
## ================= Unemployment table =================
unemployment = pd.read_csv("./data/Economy/Unemployment.csv")
#On supprime les lignes vide de Country code qui sont 1) des données non pratiquables 2) la source des données
unemployment = unemployment[unemployment['Country Code'].notna()]
#Et on supprime les colonnes Series et Series Code qui sont inutiles pour l'avenir
unemployment = unemployment.drop(['Series','Series Code'], axis=1)


In [54]:
UN_rate = unemployment.melt(id_vars=["Country","Country Code"],
                  var_name="Year",
                  value_name="UnemploymentRate")

year = UN_rate["Year"]


In [56]:
UN_rate["Year"] = UN_rate["Year"].str.slice(0,4)
UN_rate

,Country,Country Code,Year,UnemploymentRate
0,Advanced Economies,AME,2017,5.90800498983
1,Afghanistan,AFG,2017,..
2,Albania,ALB,2017,..
3,Algeria,DZA,2017,11.7
4,Angola,AGO,2017,..
...,...,...,...,...
1535,Virgin Islands (U.S.),VIR,2023,..
1536,World,WLT,2023,5.66918667967
1537,Yemen Rep.,YEM,2023,..
1538,Zambia,ZMB,2023,..
